# Проект для «Викишоп»

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Данные-для-классификации-без-использования-модели-BERT" data-toc-modified-id="Данные-для-классификации-без-использования-модели-BERT-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Данные для классификации без использования модели BERT</a></span></li></ul></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Подготовка-к-обучению" data-toc-modified-id="Подготовка-к-обучению-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовка к обучению</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# загружаем библиотеки
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)

import re
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')

from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score

from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split

RANDOM_STATE = 42

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\izara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\izara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\izara\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# Загрузка данных
url_local = 'datasets/toxic_comments.csv.csv'
url_global = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

if os.path.exists(url_local):
    data = pd.read_csv(url_local, sep = ',')
else:
    data = pd.read_csv(url_global, sep = ',')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
# пропусков нет, типы данных корректные. Посмотрим.
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
# посмотрим на балас классов:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
# не самый идеальный баланс классов. НО исследуем.

In [7]:
# дубли маловероятны, но проверим.
data.duplicated().sum()

0

In [8]:
data = data.drop(['Unnamed: 0'], axis=1)


In [9]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


#### Данные для классификации без использования модели BERT

In [10]:
# создадим список стоп-слов
import nltk
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))
print(stops)

{'it', 'hadn', 'while', 'but', "hasn't", 'was', 'don', 'yours', 'between', 'further', "hadn't", 'did', "it's", 'she', 'below', "you'd", 'no', 'we', 'why', "doesn't", 'up', "shouldn't", 'aren', 'won', 'which', 'didn', 'their', 'is', 'against', "didn't", 'what', 'because', 'when', 'doesn', 'being', 'both', 'here', 'shouldn', 'hers', 'those', 'very', 's', 'to', 'hasn', 'all', 'again', "weren't", 'yourself', 'do', 'above', 'isn', 'can', 'with', 'into', 'am', 't', 'about', 'after', "shan't", 'that', 'had', 'so', 'not', 'who', 'are', 'does', "should've", 'just', 'of', 'and', "mustn't", 'this', 'a', 'nor', 'if', 'over', "needn't", 'mustn', 'wasn', "she's", "mightn't", 'themselves', 'before', 'd', 'couldn', 'until', 'myself', 'itself', 'an', 're', 'most', 'wouldn', 'my', 'other', 'should', 'few', "wouldn't", 'the', 've', 'have', "aren't", 'then', 'out', 'haven', "couldn't", 'her', 'm', 'during', 'you', 'too', 'by', "isn't", "haven't", 'll', 'has', 'ours', 'yourselves', 'theirs', 'y', 'there', 

In [11]:
# функция для удаления списка стол-слов

def clear_text(text):
    stop_words = set(nltk_stopwords.words('english'))
    text = text.lower()
    word_list = re.sub(r"[^a-z ]", ' ', text).split()
    word_notstop_list = [w for w in word_list if not w in stop_words]
    return ' '.join(word_notstop_list)

In [12]:
data['text'] = data['text'].apply(clear_text)

In [13]:
data.head()

,text,toxic
0,explanation edits made username hardcore metal...,0
1,aww matches background colour seemingly stuck ...,0
2,hey man really trying edit war guy constantly ...,0
3,make real suggestions improvement wondered sec...,0
4,sir hero chance remember page,0


In [14]:
# Напишем функцию для сплита и лемматизации текста
lemmatizer = WordNetLemmatizer() 

def lemm_text(text):
    lemmatizer = WordNetLemmatizer()
    word_list = text.split()
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

data['text'] = data['text'].apply(lemm_text)
data.head()

,text,toxic
0,explanation edits made username hardcore metal...,0
1,aww match background colour seemingly stuck th...,0
2,hey man really trying edit war guy constantly ...,0
3,make real suggestion improvement wondered sect...,0
4,sir hero chance remember page,0


## Обучение

### Подготовка к обучению

In [15]:
# перепроверка
corpus = data['text']
corpus[1]

'aww match background colour seemingly stuck thanks talk january utc'

In [16]:
features_train, features_test, target_train, target_test = train_test_split(
                corpus, 
                data['toxic'].values, 
                test_size=0.2, 
                stratify=data['toxic'].values, 
                shuffle=True, 
                random_state=RANDOM_STATE)
# проверка
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((127433,), (31859,), (127433,), (31859,))

In [17]:
# векторизация
count_tf_idf = TfidfVectorizer()
tfidf_train = count_tf_idf.fit_transform(features_train.values.flatten())
tfidf_test = count_tf_idf.transform(features_test.values.flatten())

### LogisticRegression

In [18]:
LR_model = LogisticRegression(solver='liblinear',
                        class_weight='balanced',
                        random_state=RANDOM_STATE)

In [19]:
%%time
LR_score = cross_val_score(LR_model, tfidf_train, target_train, cv=4, scoring='f1').mean()
LR_score

CPU times: total: 5.31 s
Wall time: 4.97 s


0.7565415538827034

### LGBMClassifier 

In [20]:
LGBM_model = LGBMClassifier(n_estimators=50,
                        class_weight='balanced',
                        boosting_type='gbdt', 
                        objective='binary', 
                        random_state=RANDOM_STATE)

In [21]:
%%time
LGBM_score = cross_val_score(LGBM_model, tfidf_train, target_train, cv=4, scoring='f1').mean()
LGBM_score

CPU times: total: 3min 11s
Wall time: 1min 37s


0.7234734783610377

### DecisionTreeClassifier

In [22]:
DTC_model = DecisionTreeClassifier(class_weight='balanced',
                                   random_state=RANDOM_STATE)

In [23]:
%%time
DTC_score = cross_val_score(DTC_model,
                            tfidf_train,
                            target_train,
                            cv=4,
                            scoring='f1').mean()
DTC_score

CPU times: total: 3min 24s
Wall time: 8min 18s


0.6403531180654127

In [24]:
# Таблица для результатов обучения молелей.
model_rep = pd.DataFrame({"Model": ['LogisticRegression', 'LGBMClassifier', 'DecisionTreeClassifier'],
                          "F1": [LR_score, LGBM_score, DTC_score]
                          })
model_rep

,Model,F1
0,LogisticRegression,0.756542
1,LGBMClassifier,0.723473
2,DecisionTreeClassifier,0.640353


## Тестирование

In [25]:
# наилучший показатель в логистической регрессии. Протестируем ее.

LR_model.fit(tfidf_train, target_train)
target_test_predict = LR_model.predict(tfidf_test)
print(f1_score(target_test, target_test_predict))

0.754390779363337


## Выводы

* Произведено изучение датасета: проверка на пропуски, дисбаланс классов.
* Произведена обработка датасета: сплит, токенизация, лемматизация, загружены и удалены стоп-слова
* Обучены три модели: LogisticRegression, LGBMClassifier, DecisionTreeClassifier. Наилучее значение показала модель LogisticRegression.
* На тестировании значение f1 было более требуемого (свыше 0.75)    